# Lista de aristas doblemente conectadas
## Librerias

In [44]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

## Clases Iniciales

### Clase Vertice

In [45]:
class vertex:
    def __init__(self, x , y, name):
        self.x = x
        self.y = y
        self.name = name
        self.edge_inci = None
        self.type = None #Tipo de vertice Start,end,regular,merge o split
        
    #Agregar arista incidente
    def add_edge_inci(self,e):
        self.edge_inci = e
    
    #Relacion de orden binary -> 0 esta por abajo, -> 1 esta por arriba
    def r_order_binary(self,q):
        if self.y > q.y:
            return 0
        elif self.y == q.y:
            if self.x < q.x:
                return 0
            else:
                return 1
        else:
            return 1

    #Relacion de orden
    def r_orden(self,q):
        if self.y > q.y:
            point = self
        elif self.y == q.y:
            if self.x < q.x:
                point = self
            else:
                point = q
        else:
            point = q
        return point
    
    
    

### Clase Semi-Arista

In [46]:
class half_edge:
    def __init__(self,start,end,name):
        self.origin = start #Origen
        self.queue = end #final
        self.name = name #nombre
        self.next = None #Siguiente
        self.prev = None #Previo
        self.face = None #Cara incidente
        self.twin = twin(self) # Gemelo

    global twin
    def twin(self): #Gemelo
        return self.queue, self.origin 

    def add_next(self,n):
        self.next = n #Siguiente
    
    def add_prev(self,pv):
        self.prev = pv #Previo

    def add_face(self,f):
        self.face = f #Cara incidente    


### Clase Cara

In [47]:
class face:
    def __init__(self,n):
        self.name = n #Nombre
        self.exterior = 'None' #frontera exterior
        self.interior = 'None' #frontera interior

## Para tener en cuenta

In [48]:
# P0 : Punto arista origen
# P1 : Punto arista next origin
# P2 : Punto arista next cola
def Producto_Cruz(P0,P1,P2):
    d = ((P1.x-P0.x)*(P2.y-P0.y))-((P2.x-P0.x)*(P1.y-P0.y))
    return d

In [49]:
# P0 : Punto arista origen
# P1 : Punto arista next origin
# P2 : Punto arista next cola
def orientacion(P0,P1,P2):
    d = Producto_Cruz(P0,P1,P2)
    if d>0:
        a = True
        print("P2 se hace un giro en sentido antihoraro de P1")
    else:
        a = False
        print("P2 esta girando en sentido horario de P1")
    return a 

## Conceptos
* **Arista incidente:** Si un vértice es el punto final de una arista, entonces
decimos que el vértice y la arista son incidentes
* Una cara y una arista de su frontera son incidentes, y una cara y un vértice de su frontera son incidentes.
* Una lista de aristas doblemente conectadas contiene una lista por caras, nodos y semi-aristas.
* Recorrer una cara en el sentido contrario a las agujas del reloj.
* Si una semi-aristae tiene v como origen y w como destino, entonces su gemela Twin(e) tiene w como origen y v como destino.
* Para llegar al límite de una cara basta con almacenar un puntero en el registro de caras a una media arista arbitraria que delimite la cara.
* **límites de los agujeros de una cara:** si se recorren en sentido contrario a las agujas del reloj, la cara queda a la derecha.
* Another consequence is that twin half-edges always have opposite orientations
* Frontera interior giro en sentido antihorario
* Frontera exterior giro en setido horario

### Clase DCEL

In [50]:
class DCEL():
    # Ingresaremos unicamente la lista de objetos vertices
    def __init__(self,vertex):
        self.list_edge = []
        self.list_vertex = vertex
        self.list_face = []

    #Generar los gemelos con sus caracteristicas
    def _add_twin(self):
        twins = []
        twins_sorted = []
        edges_copy = self.list_edge.copy()
        for i in range(len(edges_copy)):
            t_start, t_end = twin(edges_copy[i])
            twins.append(half_edge(t_start,t_end,'te'+str(i+1)))

        print('Entro2')
        
        # Organizamos para que quede de manera ciclica y agremaos el next
        origen = twins[0].queue
        a = True
        while(a != False):
            edge_next = [i for i in twins  if i.origin == origen]
            twins_sorted.append(edge_next[0])
            origen = edge_next[0].queue
            if len(twins_sorted) == len(twins):
                a = False
        #for i in twins_sorted:
         #   print(i.name)
        index = len(self.list_edge) - len(twins_sorted)
        #print(len(twins_sorted))
        # Agregamos el next y prev
        for i in range(0,len(twins_sorted)+1):
            #print(i)
            #print(twins_sorted[i].name)
            if i == 0:
                #print(twins_sorted[0].name)
                self.list_edge.append(twins_sorted[0])
            else:
                #print(self.list_edge[-1].name)
                prev_twin = self.list_edge[-1]
                #print('prev_twin',prev_twin.name)
                if i == len(twins_sorted):
                    index = len(self.list_edge) - len(twins_sorted)
                    prev_twin.add_next(self.list_edge[index])
                    self.list_edge[index].add_prev(prev_twin)
                    self.list_edge[-1] = prev_twin
                    #print('next',prev_twin.next.name)
                else:
                    #print('estoy')
                    twins_sorted[i].add_prev(prev_twin)
                    #print('twin', twins_sorted[i].name)
                    #print('prevprev',twins_sorted[i].prev.name)
                    prev_twin.add_next(twins_sorted[i])
                    #print('next',prev_twin.next.name)
                    self.list_edge.append(twins_sorted[i])
        
                    

    #agregar los seguientes y previos a cada objeto de las semi aristas
    def _add_next(self):
        for i in range(len(self.list_edge)):
            if i == len(self.list_edge)-1:
                self.list_edge[i].add_next(self.list_edge[0])
            else:
                self.list_edge[i].add_next(self.list_edge[i+1])   
    
    def _add_prev(self):
        for i in range(len(self.list_edge)):
            if i == 0:
                self.list_edge[i].add_prev(self.list_edge[len(self.list_edge)-1])
            else:
                self.list_edge[i].add_prev(self.list_edge[i-1])
    
    
    
    #Generaremos la lista de aristas 
    def create_half_edge(self):
        print('Entro')
        #print(len(self.list_vertex))
        for i in range(len(self.list_vertex)): 
            name_edge = 'e'+str(i+1)
            if i == (len(self.list_vertex)-1):
                e = half_edge(self.list_vertex[i], self.list_vertex[0], name_edge)
                self.list_vertex[i].add_edge_inci(e)
            else: 
                #print(i)
                #print(i+1)
                e = half_edge(self.list_vertex[i], self.list_vertex[i+1], name_edge)
                self.list_vertex[i].add_edge_inci(e)

            self.list_edge.append(e)

        self._add_next()
        self._add_prev()
        self._add_twin()
        
    
    #El mas arriba a la izq
    def p_up(self):
        p = self.list_vertex[0]
        for i in range(len(self.list_vertex)):
            if self.list_vertex[i] == p:
                p = p.r_orden(self.list_vertex[i+1])
            else:
                p = p.r_orden(self.list_vertex[i])
        return p
    
    #generar un ciclo 
    def cicle(self,e,f):
        e.add_face(f)
        print(e.name)
        print(e.face.name)
        edge = e
        e = e.next
        while(e != edge):
            print(e.name)
            e.add_face(f)
            e = e.next

    #Generar lista de Caras
    def create_face(self):
        Inter,exter = face('I'),face('E')
        
        #Punto más arriba a la izquierda 
        p = self.p_up()
        idex = 0
        # Asegurarse de tomar la arista e que tenga ese punto como origen y que no sea la gemela
        for i in self.list_edge:
            if (p == i.origin) and (i.name == 'e'):
                idex = self.list_edge.index(i)
        
        e_inc = self.list_edge[idex]
        print(e_inc.name)
        
        start_t,end_t = twin(e_inc)
        e_twin = None
        #Encontrar el gemelo
        for i in self.list_edge:
            if (i.origin == start_t) and (i.queue == end_t):
                e_twin =i
                
        print(e_twin.next)

        a = orientacion(e_inc.origin,e_inc.next.origin,e_inc.next.queue)
        if a == True:
            Inter.interior = e_inc
            self.cicle(e_inc,Inter)
            exter.exterior = e_twin
            self.cicle(e_twin,exter)
            print('Las aristas que conformar dicho ciclo son frontera de cara interior')
        else:
            exter.exterior = e_inc
            self.cicle(e_inc,exter)
            Inter.interior = e_twin
            self.cicle(e_twin,exter)
            print('Las aristas que conformar dicho ciclo son frontera de cara exterior')
            
        self.list_face.append(Inter)
        self.list_face.append(exter)

    # A partir de los vertices vamos a generar la lista de caras y semi-aritas
    def create_edge_face(self):
        self.create_half_edge()
        self.create_face()


    

## Generar Vertice 

In [51]:
def general_vertex(P):
    list_vertex = []
    for i in range(len(P)):
        name = "V"+str(i+1)
        v = vertex(P[i][0],P[i][1], name)
        list_vertex.append(v)
    return list_vertex

## Prueba

In [52]:
P  = [[0,1], [-1,0],[0,-1],[1,0]]
lv = general_vertex(P) #genera los vertex en objetos
LADE = DCEL(lv) #lista de aristas doblemente enlazadas
LADE.create_edge_face()


Entro
Entro2
e1
P2 se hace un giro en sentido antihoraro de P1
e1
I
e2
e3
e4
te1
E
te4
te3
te2
Las aristas que conformar dicho ciclo son frontera de cara interior


# Lista veritice
La lista de nodos contienen, inicialmente las coordenadas del punto (V), defino una semi- arista incidente.
* The vertex record of a vertex v stores the coordinates of v in a field called
Coordinates(v). It also stores a pointer IncidentEdge(v) to an arbitrary
half-edge that has v as its origin.

In [53]:
for i in range(len(LADE.list_vertex)):
    print(LADE.list_vertex[i].name,LADE.list_vertex[i].edge_inci.name,[LADE.list_vertex[i].x,LADE.list_vertex[i].y])
   

V1 e1 [0, 1]
V2 e2 [-1, 0]
V3 e3 [0, -1]
V4 e4 [1, 0]


# Lista semi-aristas
* The half-edge record of a half-edge e stores a pointer Origin(e) to its origin,
a pointer Twin(e) to its twin half-edge, and a pointer IncidentFace(e) to
the face that it bounds. We don’t need to store the destination of an edge,
because it is equal to Origin(Twin(e)). The origin is chosen such that 
IncidentFace(e) lies to the left of e when it is traversed from origin to
destination. The half-edge record also stores pointers Next(e) and Prev(e)
to the next and previous edge on the boundary of IncidentFace(e). Thus
Next(e) is the unique half-edge on the boundary of IncidentFace(e) that has
the destination of e as its origin, and Prev(e) is the unique half-edge on the
boundary of IncidentFace(e) that has Origin(e) as its destination. 

In [54]:
for i in range(len(LADE.list_edge)):
    print(LADE.list_edge[i].origin.name,LADE.list_edge[i].name,LADE.list_edge[i].next.name,LADE.list_edge[i].prev.name,LADE.list_edge[i].face.name)
    

V1 e1 e2 e4 I
V2 e2 e3 e1 I
V3 e3 e4 e2 I
V4 e4 e1 e3 I
V1 te4 te3 te1 E
V4 te3 te2 te4 E
V3 te2 te1 te3 E
V2 te1 te4 te2 E


# Lista Caras
* The face record of a face f stores a pointer OuterComponent( f ) to some
half-edge on its outer boundary. For the unbounded face this pointer is nil.
It also stores a list InnerComponents( f ), which contains for each hole in
the face a pointer to some half-edge on the boundary of the hole.

In [55]:

print(LADE.list_face[0].name, 'None', LADE.list_face[0].interior.name)
print(LADE.list_face[1].name, LADE.list_face[1].exterior.name, 'None')

I None e1
E te1 None


# Triangulación de Poligonos
## Tipos de Vertices
- Start vertex: Un vertice v es *start vertex* si sus vecinos se encuentran debajo de este y el angulo interior < $\pi$ (Giro a la izquierda-Antihorario)
- End vertex: Cuando v esta por debajo de sus vecinos, angulo interior < $\pi$ (Giro derecha -AntiHorario)
- Split vertex: Si esta por encima de sus vecinos y el angulo interior >  $\pi$ (Giro izquierda-Horario)
- Merge vertex: cuando v esta por debajo de sus vecinos, angulo interior > $\pi$ (Giro derecha- Horario)
- Regula vertex: Tiene uno de sus vecinos por encima y otro por debajo. *No es vertice de giro*

*Para esta funcion va a entrar un DCEL en donde vamos a claseficar el vertice dependiendo de su arista incidente, además vamos a usar el producto cartesiano realizado arriba y la orientación. Adicionalmente utilizaremos el metodo de la clase vetex r_order_binary el cual nos va a sacar un 0 si esta por debajo y un 1 sino*

- Un punto p esta por debajo de otro punto q si

$p_y < q_y \vee ( p_y = q_y  \wedge p_x > q_x )$

- Un punto p esta por encima de otro punto q si

$p_y > q_y \vee ( p_y = q_y  \wedge p_x < q_x )$

In [62]:
# Funcion para mirar la posicion de los vecinos de un vertice en particular 
def neighbors(v):
    e = v.edge_inci # Tomamos la arista incidente
    e_prev = e.prev #Tomamos la semi-arista previa
    e_next = e.next #Tomamos la semi-arista next
    v1 = e_prev.origin #Tomamos el vertice que da origen a la semi-arista prev
    v2 = e_next.queue # Tomamos el vetice final de la semi arista next
    pos1 = v.r_order_binary(v1) #Vemos la posicion de v1 con respecto a v
    pos2 = v.r_order_binary(v2) #Vemos la posicion de V2 con respecto a v
    print(pos1,pos2)
    print('v1:',v1.x,v1.y)
    print('v2:',v2.x,v2.y)
    if pos1 != pos2:
        print('uno arriba y otro abajo')
        return 1 #Esta uno arriba y otro abajo
    else:
        if pos1 == 1 and pos2 == 1:
            print('Ambos estan arriba')
            return 2 #Ambos estan abajo
        elif pos1 == 0 and pos2 == 0:
            print('Ambos estan abajo')
            return 3 #Ambos estan abajo


In [74]:
def type_vertex(v): 
    pos = neighbors(v)
    if pos == 3: #Estan por debajo
        if orientacion(v.edge_inci.prev.origin,v.edge_inci.origin,v.edge_inci.next.queue): #TRUE - antihorario
            v.type = 'start'
        else: #False - horario
            v.type = 'split'
    elif pos == 2: #Estan por arriba
        if orientacion(v.edge_inci.prev.origin,v.edge_inci.origin,v.edge_inci.next.queue): #TRUE - antihorario
            v.type = 'end'
        else: #False - horario
            v.type = 'merge'
    else:
        v.type = 'regular'
    return v
    #print(v1.type)

### Prueba

In [78]:
vertex = LADE.list_vertex
v = LADE.list_vertex[3]
LADE.list_vertex[0] = type_vertex(v)
print(LADE.list_vertex[0].type)

0 1
v1: 0 -1
v2: -1 0
uno arriba y otro abajo
regular


## Algoritmo de Insertar una Diagonal

In [ ]:
#Ingresamos dos semi-aristas, donde k no se halla usado anteriormente. deben ser objetos half-edge
# El va va a ser el origen de ek1 y vb va ser el origen de ek2
def insert_diagonal(DCEL,ek1,ek2,va,vb):
    # 1. el k no se debio de haber usado antes.
    # 2. asigno el origen y el gemelo.
    ek1.origin = va
    ek2.origin = vb
    # 3. Identifico las semi-aristas ea y eb que tienen origen en va y vb respectivamente, que tienen la misma cara incidente 
    # y que estan en un ciclo de forntera exterior

